In [ ]:
import pandas as pd
from netCDF4 import Dataset
from wrf import getvar, latlon_coords, to_np, ll_to_xy, Constants
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import cartopy.feature as cfeature
import xarray as xr

In [ ]:
# Preprocessing of TITAN data
df = pd.read_csv("/scratch3/tim/wrf_4.0.1/P3/tracks/tracks_20180531.csv.gz", sep=",", header=41)

# Selcting cells lasting longer than 10 min (=600s)
df_duration_600 = df[df["stormDuration"] > 600]

# Selcting cells initiated over Switzerland [5.9, 10.5, 45.75, 47.85]
df_ch_min = df_duration_600[(df_duration_600['xCoord'] > 5.9) & (df_duration_600['yCoord'] > 45.75)]
df_ch_max = df_ch_min[(df_ch_min['xCoord'] < 10.5) & (df_ch_min['yCoord'] < 47.85)]

# Selcting cells with no parents
df_np = df_ch_max[df_ch_max['nParents'] == 0]

# Reset index
df_np = df_np.reset_index()

In [ ]:
# Selecting cells with highest maxZ values corresponding to sorting before
df_np_maxZ = df_ch_max[(df_ch_max["maxZ"]>57.4) & (df_ch_max["maxZ"]<57.5)]

print(df_np_maxZ[['stormID', 'trackID', 'originTime', 'timestamp', "maxZ", 
                                'xCoord', 'yCoord', 'nParents', 'lowdBZThreshold']])

# Check highest maxZ values
df_maxZ = df_np_maxZ["maxZ"].sort_values()

print(df_maxZ)

In [ ]:
# Selecting cells with highest maxZ values corresponding to sorting before
df_np_maxZ = df_np[(df_np["maxZ"]>57.4) & (df_np["maxZ"]<57.5)]

print(df_np_maxZ[['stormID', 'trackID', 'originTime', 'timestamp', "maxZ", 'index', 
                                'xCoord', 'yCoord', 'nParents', 'lowdBZThreshold']])

# Check highest maxZ values
df_maxZ = df_np_maxZ["maxZ"].sort_values()

print(df_maxZ)

In [ ]:
stormID_1 = df_np[df_np['stormID']==17430] # maxZ = 58.3960 @ time step 1
case_study_1 = df_np[df_np['index']==86737]

stormID_2 = df_np[df_np['stormID']==4630] # maxZ  = 57.8741 @ time step 1
case_study_2 = df_np[df_np['index']==22572]

stormID_3 = df_np[df_np['stormID']==6361] # maxZ  = 57.5519 @ time step 1
s3_ts = stormID_3[stormID_3['originTime']== '2018-05-12T20:15:00']
case_study_3 = df_np[df_np['index']==31923]

stormID_4 = df_np[df_np['stormID']==5242] # maxZ  = 57.4788 @ time step 4 (20 min after)
s4_ts = stormID_4[stormID_4['originTime']== '2018-05-10T01:00:00']
s4_maxZ = s4_ts["maxZ"].sort_values()
case_study_4 = df_np[df_np['index']==26206]

stormID_5 = df_np[df_np['stormID']==27749] # maxZ  = 57.4781 @ time step 1
s5_ts = stormID_5[stormID_5['originTime']== '2018-05-30T14:50:00']
case_study_5 = df_np[df_np['index']==134068]

case_study = case_study_3
print(case_study[['stormID', 'trackID', 'xCoord', 'yCoord', 'index', 'stormDuration', 
             'timestamp', 'originTime', 'maxZ', 'nParents', 'nChildren']])

## Quick plots of the case study locations

In [ ]:
# Make a function that deals with points which have longitudes > STAND_LON
def rotated_ll_to_xy(wrfFile, latitude, longitude):
    res = ll_to_xy(wrfFile, latitude=latitude, longitude=longitude)
    diffPix = np.round((2*Constants.WRF_EARTH_RADIUS*Constants.PI)/wrfFile.DX,0)
    xs = np.where(np.ravel(longitude) >= wrfFile.STAND_LON*-1, res[0]+diffPix, res[0])
    res[0] = xs[0]
    return(res)

filename = '/scratch3/thomasl/work/data/case_study_1/' \
                'wrfout_d02_2018-05-22_17:00:00'

initiation_location = [case_study.xCoord, case_study.yCoord]

ncfile = Dataset(filename)
wrf_file = xr.open_dataset(filename)
data = wrf_file.HGT

location = rotated_ll_to_xy(ncfile, initiation_location[1], initiation_location[0])
data_point = data.sel(west_east=location[0], south_north=location[1])
hgt = data_point.values
print('Terrain height: {} m'.format(hgt))

In [ ]:
# Plot all initiations selected before
wrfFile = Dataset('/scratch3/thomasl/work/data/case_study_4/wrfout_d02_2018-05-10_01:10:00')

ter = getvar(wrfFile, 'ter')

lats, lons = latlon_coords(ter)

cart_proj = ccrs.LambertConformal(central_longitude=8.722206, 
                                    central_latitude=46.73585)

fig = plt.figure(figsize=(15,10))
ax = plt.axes(projection=cart_proj)

ax.set_extent([5.9, 10.5, 45.75, 47.85])

ax.contourf(to_np(lons), to_np(lats), to_np(ter), transform=ccrs.PlateCarree())

for i in s5_ts[['index','xCoord','yCoord']].to_numpy():
    ax.plot(i[1], i[2], 'wx', zorder=7, transform=ccrs.PlateCarree(), markersize=15)
    ax.text(i[1], i[2], int(i[0]), zorder=7, transform=ccrs.PlateCarree(), fontsize=15, color='w')
    
ax.add_feature(cfeature.BORDERS.with_scale('10m'), zorder=5, 
               linewidth=1.5)
plt.show()

In [ ]:
# Plot all initiations selected before
wrfFile = Dataset('/scratch3/thomasl/work/data/case_study_1/wrfout_d02_2018-05-10_01:00:00')

ter = getvar(wrfFile, 'ter')

lats, lons = latlon_coords(ter)

cart_proj = ccrs.LambertConformal(central_longitude=8.722206, 
                                    central_latitude=46.73585)

fig = plt.figure(figsize=(15,10))
ax = plt.axes(projection=cart_proj)

ax.set_extent([5.9, 10.5, 45.75, 47.85])

ax.contourf(to_np(lons), to_np(lats), to_np(ter), transform=ccrs.PlateCarree())
    
ax.plot(case_study_1['xCoord'], case_study_1['yCoord'], 'w*', zorder=7, transform=ccrs.PlateCarree(), markersize=15)
ax.text(case_study_1['xCoord']+0.05, case_study_1['yCoord'], 'Case Study I', zorder=7, transform=ccrs.PlateCarree(), fontsize=15, color='w')

ax.plot(case_study_2['xCoord'], case_study_2['yCoord'], 'w*', zorder=7, transform=ccrs.PlateCarree(), markersize=15)
ax.text(case_study_2['xCoord']+0.05, case_study_2['yCoord'], 'Case Study II', zorder=7, transform=ccrs.PlateCarree(), fontsize=15, color='w')

ax.plot(case_study_3['xCoord'], case_study_3['yCoord'], 'w*', zorder=7, transform=ccrs.PlateCarree(), markersize=15)
ax.text(case_study_3['xCoord']+0.05, case_study_3['yCoord'], 'Case Study III', zorder=7, transform=ccrs.PlateCarree(), fontsize=15, color='w')

ax.plot(case_study_4['xCoord'], case_study_4['yCoord'], 'w*', zorder=7, transform=ccrs.PlateCarree(), markersize=15)
ax.text(case_study_4['xCoord']+0.05, case_study_4['yCoord'], 'Case Study IV', zorder=7, transform=ccrs.PlateCarree(), fontsize=15, color='w')

ax.plot(case_study_5['xCoord'], case_study_5['yCoord'], 'w*', zorder=7, transform=ccrs.PlateCarree(), markersize=15)
ax.text(case_study_5['xCoord']+0.05, case_study_5['yCoord'], 'Case Study V', zorder=7, transform=ccrs.PlateCarree(), fontsize=15, color='w')

ax.add_feature(cfeature.BORDERS.with_scale('10m'), zorder=5, 
               linewidth=1.5)
plt.show()

#fig.savefig('initiations_30738', bbox_inches='tight', dpi=300)